In [ ]:
import xarray as xr
import numpy as np

# Load the NetCDF file
input_nc_file = "D:/Data/NSOAS/MUL/MUL_OPER_nrt_global_allsat_phy_20241215_20241217_T.nc"
output_directory = "D:/Data/NSOAS/MUL/test"

# Specify the variables to extract
variables_to_extract = ['sla', 'err', 'adt', 'u', 'ua', 'v', 'va']

# Load the dataset
ds = xr.open_dataset(input_nc_file)

# Iterate over variables and save each one as a separate file
for var in variables_to_extract:
    if var in ds.variables:
        # Extract the variable as a DataArray
        var_data = ds[var]

        # Add coordinates to the DataArray if needed (usually already included)
        var_data.coords['lat'] = ds['lat']
        var_data.coords['lon'] = ds['lon']

        # Copy attributes from the original variable
        var_data.attrs = ds[var].attrs
        var_data = var_data.where(var_data != -214748.3647, np.nan)

        # Define the output file path
        output_file = f"{output_directory}/{var}.nc"

        # Save the extracted variable to a new NetCDF file without encoding
        var_data.to_netcdf(output_file)
        print(f"Saved {var} to {output_file}")
    else:
        print(f"Variable {var} not found in the dataset.")

In [1]:
import os
import re
import xarray as xr
import numpy as np
from datetime import datetime

def scan_and_process_latest_nc_file(base_path, output_base_path, variables_to_extract):
    # 正则表达式匹配文件名并提取第一个日期
    file_pattern = re.compile(r"MUL_OPER_nrt_global_allsat_phy_(\d{8})_\d{8}_T\.nc$")
    
    latest_file = None
    latest_date = None

    for root, _, files in os.walk(base_path):
        for file in files:
            if file.startswith("MUL_OPER_nrt_global_allsat_phy") and file.endswith(".nc"):
                match = file_pattern.match(file)
                if match:
                    file_date = match.group(1)
                    try:
                        # 转换为 datetime 对象
                        date_obj = datetime.strptime(file_date, "%Y%m%d")
                        file_path = os.path.join(root, file)

                        # 更新为最新日期文件
                        if latest_date is None or date_obj > latest_date:
                            latest_date = date_obj
                            latest_file = file_path
                    except ValueError:
                        print(f"Invalid date format in file: {file}")

    # 如果找到了最新文件，则处理它
    if latest_file:
        print(f"Latest file: {latest_file}, Date: {latest_date.strftime('%Y-%m-%d')}")
        process_nc_file(latest_file, output_base_path, latest_date, variables_to_extract)
    else:
        print("No valid files found.")

def process_nc_file(input_nc_file, output_base_path, date_obj, variables_to_extract):
    # 加载 NetCDF 文件
    ds = xr.open_dataset(input_nc_file)

    # 提取日期信息
    year = date_obj.year
    month = date_obj.month
    day = date_obj.day

    for var in variables_to_extract:
        if var in ds.variables:
            # 提取变量
            var_data = ds[var]
            # 添加坐标
            var_data.coords['lat'] = ds['lat']
            var_data.coords['lon'] = ds['lon']
            # 复制属性
            var_data.attrs = ds[var].attrs
            # 替换无效值为 NaN
            var_data = var_data.where(var_data != -214748.3647, np.nan)
            
            # 定义输出路径
            output_dir = f"{output_base_path}/{year}/{month}/{day}/{var}_nsoas"
            os.makedirs(output_dir, exist_ok=True)  # 创建目录
            output_file = f"{output_dir}/data.nc"

            # 保存变量到新文件
            var_data.to_netcdf(output_file)
            print(f"Saved {var} to {output_file}")
        else:
            print(f"Variable {var} not found in the dataset.")

# 主函数
if __name__ == "__main__":
    # 输入路径
    base_path = "/datadisk1/2024"
    # 输出路径
    output_base_path = "/data/fishdata"
    # 需要提取的变量
    variables_to_extract = ['sla', 'adt', 'u', 'ua', 'v', 'va']
    
    # 扫描并处理最新文件
    scan_and_process_latest_nc_file(base_path, output_base_path, variables_to_extract)


No valid files found.


In [7]:
import xarray as xr
ds = xr.open_dataset("C:/datadisk1/2024/12/15/MUL_OPER_SST_L4A_FU_01D_20241215_dps_050_10_sst.nc")
print(ds)
lon = np.arange(1/40, 360 - 1/40, 1/20)
lat = np.arange(-90 + 1/40, 90, 1/20)
lat = np.flip(lat)
print(len(lon))
print(len(lat))

<xarray.Dataset> Size: 104MB
Dimensions:  (x: 3600, y: 7200)
Dimensions without coordinates: x, y
Data variables:
    SST      (x, y) float32 104MB ...
7200
3600


In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import matplotlib.ticker as mticker
import matplotlib.patches as patches

# Load the NetCDF file
ds = xr.open_dataset('D:/Data/NSOAS/MUL/test/sla.nc')

# Read the 'sla' variable
sla_data = ds['sla']

# Replace FillValue with NaN
# sla_data = sla_data.where(sla_data != -214748.3647, np.nan)

# Read latitude and longitude
latitudes = ds['lat'].values
longitudes = ds['lon'].values

# Adjust latitudes and longitudes to match pcolormesh requirements
latitudes = np.append(latitudes, latitudes[-1] + (latitudes[-1] - latitudes[-2]))
longitudes = np.append(longitudes, longitudes[-1] + (longitudes[-1] - longitudes[-2]))

lon_grid, lat_grid = np.meshgrid(longitudes, latitudes)

# Set plot parameters
plt.rcParams['font.size'] = 12
plt.rcParams['axes.linewidth'] = 1
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['xtick.minor.visible'] = True
plt.rcParams['ytick.minor.visible'] = True
plt.rcParams['figure.dpi'] = 200
plt.rcParams['savefig.dpi'] = 200
plt.rcParams['savefig.bbox'] = 'tight'
plt.rcParams['savefig.pad_inches'] = 0.1
plt.rcParams['font.family'] = 'Times New Roman'

# Create the figure and axis
fig = plt.figure(figsize=(12, 6), dpi=200)
plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.15)
ax1 = plt.axes(projection=ccrs.PlateCarree(central_longitude=180))


# Plot the SLA data
c = ax1.pcolormesh(lon_grid, lat_grid, sla_data, shading='flat', cmap='coolwarm', vmin=-0.2, vmax=0.2, transform=ccrs.PlateCarree())

# Add land and borders
ax1.add_feature(cfeature.LAND, facecolor='lightgrey')
ax1.add_feature(cfeature.BORDERS, edgecolor='black', linewidth=1)

# Add colorbar
cbar = fig.colorbar(c, ax=ax1, orientation='vertical', pad=0.02, shrink=0.75, aspect=40)
cbar.set_label("Sea Level Anomaly (m)", fontsize=12)
cbar.ax.yaxis.set_label_position('right')
cbar.minorticks_off()
cbar.ax.tick_params(length=6)

# Set title and labels
ax1.set_title("Sea Level Anomaly (SLA)", fontsize=14)
ax1.set_xlabel("Longitude (°)", fontsize=12)
ax1.set_ylabel("Latitude (°)", fontsize=12)

# Add coastlines
ax1.coastlines()

# Set ticks and formatters
ax1.set_xticks(np.arange(-180, 181, 60), crs=ccrs.PlateCarree())
ax1.set_yticks(np.arange(-90, 91, 30), crs=ccrs.PlateCarree())
ax1.xaxis.set_major_formatter(LongitudeFormatter())
ax1.yaxis.set_major_formatter(LatitudeFormatter())
ax1.xaxis.set_major_locator(mticker.MultipleLocator(60))
ax1.xaxis.set_minor_locator(mticker.AutoMinorLocator(4))

# Add gridlines
gridlines = ax1.gridlines(draw_labels=False, linestyle='--', linewidth=0.5, color='grey')

# Draw a rectangle (example coordinates, adjust as needed)
rect = patches.Rectangle(xy=(160, -40), width=120, height=40, linewidth=1, edgecolor='red', facecolor='none', transform=ccrs.PlateCarree())
ax1.add_patch(rect)

# Save the figure
output_path = "D:/Data/NSOAS/MUL/test/sla_visualization.png"
plt.savefig(output_path, format='png')

plt.tight_layout()
plt.show()

In [ ]:
print(sla_data.min().item())